<a href="https://colab.research.google.com/github/dsynderg/CS-479-machine-translation/blob/main/my_mtw_sentancepiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Download data
!wget https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
!tar xf toy-ende.tar.gz

--2025-10-07 23:46:18--  https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.198.128, 3.5.3.251, 52.216.56.96, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.198.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1662081 (1.6M) [application/x-gzip]
Saving to: ‘toy-ende.tar.gz’

toy-ende.tar.gz     100%[===================>]   1.58M  4.33MB/s    in 0.4s    

2025-10-07 23:46:19 (4.33 MB/s) - ‘toy-ende.tar.gz’ saved [1662081/1662081]



In [31]:
!pip install sentencepiece

In [32]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
  input='src_train.txt',
  model_prefix='english_sp_model',
  vocab_size=16000,
)

spm.SentencePieceTrainer.train(
  input='tgt_train.txt',
  model_prefix='spanish_sp_model',
  vocab_size=16000,
)

In [2]:
!pip install OpenNMT-py
!pip install "numpy<2.0" # This fixes an error caused by OpenNMT not being maintained

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [33]:
# Build YAML Config file
yaml_text = """
# sentance_peice_yaml.yaml

## Where the samples will be written
save_data: /content/example

src_vocab: /content/example.vocab.src
tgt_vocab: /content/example.vocab.tgt

# Prevent overwriting existing files in the folder
overwrite: True

# -- Sentencepiece Params --
# Tokenization options
src_subword_type: sentencepiece
src_subword_model: english_sp_model.model
tgt_subword_type: sentencepiece
tgt_subword_model: spanish_sp_model.model

# Number of candidates for SentencePiece sampling
subword_nbest: 20
# Smoothing parameter for SentencePiece sampling
subword_alpha: 0.1
# Specific arguments for pyonmttok
src_onmttok_kwargs: "{'mode': 'none', 'spacer_annotate': True}"
tgt_onmttok_kwargs: "{'mode': 'none', 'spacer_annotate': True}"


# Corpus opts:
data:
    corpus_1:
        path_src: /content/src_train.txt
        path_tgt: /content/tgt_train.txt
        transforms: [onmt_tokenize]
    valid:
        path_src: /content/src_val.txt
        path_tgt: /content/tgt_val.txt
        transforms: [onmt_tokenize]

# Train on a single GPU
bucket_size: 10000
world_size: 1
gpu_ranks: [0]

# Where to save the checkpoints
save_model: /content/modle
save_checkpoint_steps: 500
train_steps: 20000
valid_steps: 500
"""

with open("sentance_peice_yaml.yaml", "w") as f:
    f.write(yaml_text)

In [36]:
!onmt_build_vocab -config sentance_peice_yaml.yaml

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-10-07 23:48:18,774 INFO] Counter vocab from 5000 samples.
[2025-10-07 23:48:18,774 INFO] Build vocab on 5000 transformed examples/corpus.
[2025-10-07 23:48:19,492 INFO] Counters src: 10052
[2025-10-07 23:48:19,492 INFO] Counters tgt: 11203
[2025-10-07 23:48:19,492 WARNING] path /content/example.vocab.src exists, may overwrite...
[2025-10-07 23:48:19,500 WARNING] path /content/example.vocab.tgt exists, may overwrite...


In [37]:
!onmt_train -config sentance_peice_yaml.yaml

[2025-10-07 23:48:54,089 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-10-07 23:48:54,089 INFO] Parsed 2 corpora from -data.
[2025-10-07 23:48:54,090 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2025-10-07 23:48:54,125 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁the', '.', '▁and', '▁of', '▁to']
[2025-10-07 23:48:54,125 INFO] The decoder start token is: <s>
[2025-10-07 23:48:54,126 INFO] Building model...
[2025-10-07 23:48:58,725 INFO] Switching model to float32 for amp/apex_amp
[2025-10-07 23:48:58,725 INFO] Non quantized layer compute is fp32
[2025-10-07 23:48:59,053 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(10056, 500, padding_idx=1)
        )
      )
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (rnn): LSTM(500, 500, num_layers=2, batch_first=True, 

In [40]:
translate_yaml_text = """
model: /content/modle_step_20000_sentance_peice.pt
src: /content/src_test.txt
output: /content/sentancepeice_2000.txt
gpu: 0
verbose: True

# Sentencepiece Params

# Tokenization options
transforms: onmt_tokenize
src_subword_type: sentencepiece
src_subword_model: english_sp_model.model
tgt_subword_type: sentencepiece
tgt_subword_model: spanish_sp_model.model

src_subword_nbest: 20
src_subword_alpha: 0.1
"""

with open("translate_yaml.yaml", "w") as f:
    f.write(translate_yaml_text)

In [41]:
!onmt_translate -config translate_yaml.yaml

Streaming output truncated to the last 5000 lines.
PRED 1001: ▁Debe ▁hacer ▁una ▁diferencia ▁en ▁la ▁forma ▁en ▁la ▁que ▁seas ▁y ▁en ▁las ▁cosas ▁que ▁hacen ▁y ▁las ▁decisiones ▁que ▁hacen .
PRED SCORE: -0.5823

[2025-10-08 00:37:21,169 INFO] 
SENT 1002: ['▁Come', '▁back', '▁during', '▁the', '▁conference', '▁to', '▁see', '▁if', '▁more', '▁of', '▁your', '▁relatives', '▁have', '▁join', 'ed', '.']
PRED 1002: ▁Regresa ▁durante ▁la ▁conferencia ▁para ▁ver ▁si ▁se ▁ha n ▁unido ▁más ▁de ▁tu s ▁parientes .
PRED SCORE: -0.2275

[2025-10-08 00:37:21,169 INFO] 
SENT 1003: ['▁', 'Do', 'es', '▁this', '▁lesson', '▁have', '▁information', '▁referr', 'ing', '▁to', '▁the', '▁temple', '?']
PRED 1003: ▁¿ Esta ▁lección ▁tiene ▁información ▁referente ▁al ▁templo ?
PRED SCORE: -0.5077

[2025-10-08 00:37:21,170 INFO] 
SENT 1004: ['▁Living', '▁Sacred', '<unk>', 's', '▁Before', '▁', 'the', '▁Lord']
PRED 1004: ▁La ▁vida ▁sagrada ▁va ▁al ▁Señor
PRED SCORE: -0.9237

[2025-10-08 00:37:21,170 INFO] 
SENT 1005: ['▁Th